In [ ]:
# Pandas and Python
import pandas as pd
pd.options.display.float_format = '{:.4f}'.format
import numpy as np
from tqdm import tqdm
import os
import time
from functools import partial
from multiprocessing import Pool

# Graphic Libraries
import plotly.io as pio
pio.templates.default = "simple_white"
pd.options.plotting.backend = "plotly"
import matplotlib as plt
from IPython.display import display, HTML, clear_output


# AI and stats
import statsmodels.api as sm
import xgboost
from xgboost import XGBRegressor
import torch
import sklearn




In [3]:
os.getcwd()

'/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/scripts'

In [7]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/")

# Main data
ff3_mom_daily = pd.read_csv("3F+MoM_daily.csv")
ff5_mom_daily = pd.read_csv("5F+MoM_daily.csv")
portfolios_5x5_bm_size =  pd.read_csv("5x5ResearchPortfolios.csv")
ff_industry_12_mapping = pd.read_csv("FF_industry12descriptions.csv")
ff_industry_48_mapping = pd.read_csv("FF_indutry48descriptions.csv")
ff5_portfolio_returns = pd.read_csv("FF5Portfolios_dailyRet.csv")
ff3_monthly_factors = pd.read_csv("FFResearchF_monthly.csv")
pastor_stambaugh_liq = pd.read_csv("P+SLiquidityfactors.csv")
riskfree_rate_daily = pd.read_csv("riskfreerate_daily.csv")

In [ ]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/fundamentals/")

monthly_val_APPL = pd.read_csv("AAPL_monthly_valuation_measures.csv")
monthly_val_AMZN = pd.read_csv("AMZN_monthly_valuation_measures.csv")
monthly_val_AVGO = pd.read_csv("AVGO_monthly_valuation_measures.csv")
monthly_val_BRKB = pd.read_csv("BRK-B_monthly_valuation_measures.csv")
monthly_val_GOOG = pd.read_csv("GOOG_monthly_valuation_measures.csv")
monthly_val_GOOGL = pd.read_csv("GOOGL_monthly_valuation_measures.csv")
monthly_val_META = pd.read_csv("META_monthly_valuation_measures.csv")
monthly_val_MSFT = pd.read_csv("MSFT_monthly_valuation_measures.csv")
monthly_val_NVDA = pd.read_csv("NVDA_monthly_valuation_measures.csv")
monthly_val_TSLA= pd.read_csv("TSLA_monthly_valuation_measures.csv")


In [8]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/")

In [9]:
# Load all sheets into a dictionary of DataFrames

xls = pd.read_excel("AAPL_options_chain.xlsx", sheet_name=None)

# List available sheet names
print("Sheets found:", list(xls.keys()))

# Access a specific sheet
df_calls = xls['Calls'] if 'Calls' in xls else None
df_puts = xls['Puts'] if 'Puts' in xls else None

# Preview
if df_calls is not None:
    print("Calls Sheet Head:")
    print(df_calls.head())

if df_puts is not None:
    print("Puts Sheet Head:")
    print(df_puts.head())


Sheets found: ['Jun 20, 2025', 'Jun 27, 2025', 'Jul 3, 2025', 'Jul 11, 2025', 'Jul 18, 2025', 'Jul 25, 2025', 'Aug 1, 2025', 'Aug 15, 2025', 'Sep 19, 2025', 'Oct 17, 2025', 'Nov 21, 2025', 'Dec 19, 2025', 'Jan 16, 2026', 'Feb 20, 2026', 'Mar 20, 2026', 'Jun 18, 2026', 'Sep 18, 2026', 'Dec 18, 2026', 'Jan 15, 2027', 'Jun 17, 2027', 'Dec 17, 2027']


In [10]:

input_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/"
output_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/"
os.makedirs(output_dir, exist_ok=True)

xlsx_files = [f for f in os.listdir(input_dir) if f.endswith(".xlsx")]
# Process each Excel file
for filename in xlsx_files:
    file_path = os.path.join(input_dir, filename)
    ticker = filename.split("_")[0].upper()  # Extract ticker from filename prefix

    try:
        # Load all sheets preserving raw content
        all_sheets = pd.read_excel(file_path, sheet_name=None, dtype=object)

        for sheet_name, df in all_sheets.items():
            safe_sheet = sheet_name.strip().lower().replace(" ", "_")
            output_path = os.path.join(output_dir, f"{ticker}_options_{safe_sheet}.csv")
            df.to_csv(output_path, index=False, encoding='utf-8')
            print(f"Saved: {output_path} ({df.shape[0]} rows, {df.shape[1]} columns)")

    except Exception as e:
        print(f"Error processing {filename}: {e}")

print("\nBatch processing completed.")


Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jun_20,_2025.csv (140 rows, 11 columns)
Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jun_27,_2025.csv (84 rows, 11 columns)
Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jul_3,_2025.csv (73 rows, 11 columns)
Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jul_11,_2025.csv (46 rows, 11 columns)
Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jul_18,_2025.csv (99 rows, 11 columns)
Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/split_by_sheet/GOOGL_options_jul_25,_2025.csv (51 rows, 11 columns)
Save

In [13]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/")

In [14]:
from glob import glob

root_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals"
output_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged"
os.makedirs(output_dir, exist_ok=True)

# Loop over subfolders (each representing a ticker)
for ticker_folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, ticker_folder)
    if not os.path.isdir(folder_path):
        continue

    # Find all *_Y_day.csv files
    year_files = sorted(glob(os.path.join(folder_path, f"{ticker_folder}_*Y_day.csv")))
    if not year_files:
        print(f"No year files found for {ticker_folder}")
        continue

    print(f"Merging {len(year_files)} files for {ticker_folder}...")

    # Load and concatenate all year files
    dfs = []
    for file in year_files:
        try:
            df = pd.read_csv(file, dtype=str)
            df['source_file'] = os.path.basename(file)  # Optional: track origin
            dfs.append(df)
        except Exception as e:
            print(f"Error reading {file}: {e}")

    if not dfs:
        print(f"No readable data for {ticker_folder}")
        continue

    full_df = pd.concat(dfs, ignore_index=True)

    # Output merged file
    merged_path = os.path.join(output_dir, f"{ticker_folder}_2014_2024_technical.csv")
    full_df.to_csv(merged_path, index=False)
    print(f"Saved merged file: {merged_path} ({full_df.shape[0]} rows)")


Merging 11 files for AMZN...
Saved merged file: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged/AMZN_2014_2024_technical.csv (2779 rows)
Merging 11 files for AAPL...
Saved merged file: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged/AAPL_2014_2024_technical.csv (2779 rows)
Merging 11 files for GOOG...
Saved merged file: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged/GOOG_2014_2024_technical.csv (2779 rows)
Merging 11 files for BRK-B...
Saved merged file: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged/BRK-B_2014_2024_technical.csv (2779 rows)
Merging 11 files for TSLA...
Saved merged file: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals/technicals_merged/TSLA_2014_2024_technical.csv (2779

In [27]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/technicals")

In [35]:
import os
import pandas as pd

def load_technicals(local_filename):
    df = pd.read_csv(local_filename)
    df.columns = [c.strip().lower() for c in df.columns]
    df.rename(columns={'date': 'date'}, inplace=True)  # Redundant, but safe
    df['date'] = pd.to_datetime(df['date'], errors='coerce')
    df = df.dropna(subset=['date'])
    df = df.sort_values('date').drop_duplicates('date')
    
    print(f"✅ Loaded {local_filename} — {df.shape[0]} rows")
    return df

# 🧠 MAIN EXECUTION
technicals_dir = "technicals_merged"
all_files = [f for f in os.listdir(technicals_dir) if f.endswith(".csv")]

all_technicals = {}

for file in all_files:
    filepath = os.path.join(technicals_dir, file)
    try:
        df = load_technicals(filepath)
        ticker = file.split("_")[0].upper()
        all_technicals[ticker] = df
        
        # 🔧 Patch: ensure correct cleaned filename
    
        cleaned_filename = file.replace(".csv", "_cleaned.csv")
        output_path = os.path.join(technicals_dir, cleaned_filename)
        print(f"💾 Saving cleaned file to: {output_path}")
        df.to_csv(output_path, index=False)

    except Exception as e:
        print(f"❌ Error loading {file}: {e}")


✅ Loaded technicals_merged/GOOGL_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/GOOGL_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/AMZN_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/AMZN_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/MSFT_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/MSFT_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/NVDA_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/NVDA_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/TSLA_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/TSLA_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/GOOG_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technicals_merged/GOOG_2014_2024_technical_cleaned.csv
✅ Loaded technicals_merged/BRK-B_2014_2024_technical.csv — 2769 rows
💾 Saving cleaned file to: technical

In [32]:
output_path = os.path.join(technicals_dir, f"{ticker}_cleaned.csv")
df.to_csv(output_path, index=False)


In [51]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/")

In [56]:
os.getcwd()

'/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options'

In [58]:
import os
import pandas as pd
from datetime import datetime
from openpyxl.utils.datetime import from_excel

# === CONFIG ===
input_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/"
output_dir = "options_cleaned_from_xlsx"
os.makedirs(output_dir, exist_ok=True)

def merge_ordered_xlsx(ticker_file, output_dir):
    ticker = os.path.basename(ticker_file).split("_")[0].upper()

    try:
        xls = pd.ExcelFile(ticker_file, engine="openpyxl")
    except Exception as e:
        print(f"❌ Failed to read {ticker_file}: {e}")
        return

    merged = []

    for sheet_name in xls.sheet_names:
        try:
            df = pd.read_excel(xls, sheet_name=sheet_name, engine="openpyxl", skiprows=2)
            df.columns = [str(c).strip().lower().replace(" ", "_") for c in df.columns]
            df = df.dropna(how='all')  # Drop rows with all NaNs

            # Save sheet info
            df["expiry_sheet"] = sheet_name.strip()

            # Parse expiry date from sheet name if possible (e.g., "Jun 20, 2025")
            try:
                expiry_dt = datetime.strptime(sheet_name.strip(), "%b %d, %Y")
            except Exception:
                expiry_dt = pd.NaT
            df["expiry_date"] = expiry_dt

            merged.append(df)

        except Exception as e:
            print(f"⚠️ Skipping sheet '{sheet_name}' in {ticker_file}: {e}")

    if merged:
        full_df = pd.concat(merged, ignore_index=True)
        outname = os.path.join(output_dir, f"{ticker}_options_all.csv")
        full_df.to_csv(outname, index=False)
        print(f"✅ Merged {ticker_file} — saved to {outname} with {full_df.shape[0]} rows")
    else:
        print(f"⚠️ No usable data in {ticker_file}")

# === MAIN LOOP ===
for file in os.listdir(input_dir):
    if file.endswith("_options_chain.xlsx"):
        full_path = os.path.join(input_dir, file)
        merge_ordered_xlsx(full_path, output_dir)


❌ Failed to read /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/~$AAPL_options_chain.xlsx: File is not a zip file
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/GOOGL_options_chain.xlsx — saved to options_cleaned_from_xlsx/GOOGL_options_all.csv with 1858 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/TSLA_options_chain.xlsx — saved to options_cleaned_from_xlsx/TSLA_options_all.csv with 4421 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/SPY_options_chain.xlsx — saved to options_cleaned_from_xlsx/SPY_options_all.csv with 7904 rows


/var/folders/g9/jl77b7s511d2m50h489cqhwr0000gp/T/ipykernel_8745/1575427794.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(merged, ignore_index=True)


✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/AVGO_options_chain.xlsx — saved to options_cleaned_from_xlsx/AVGO_options_all.csv with 3546 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/GOOG_options_chain.xlsx — saved to options_cleaned_from_xlsx/GOOG_options_all.csv with 1484 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/META_options_chain.xlsx — saved to options_cleaned_from_xlsx/META_options_all.csv with 4698 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/NVDA_options_chain.xlsx — saved to options_cleaned_from_xlsx/NVDA_options_all.csv with 6289 rows
✅ Merged /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/AAPL_options_chain.xlsx — saved to options_cleaned_from_xlsx/AAPL_options_all.csv with 2238 rows
✅ Merged /Users

/var/folders/g9/jl77b7s511d2m50h489cqhwr0000gp/T/ipykernel_8745/1575427794.py:44: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat(merged, ignore_index=True)


In [59]:
os.getcwd()

'/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options'

In [66]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors")

In [68]:
os.getcwd()

'/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors'

In [71]:
import os
import pandas as pd
from functools import reduce

# === Setup ===
input_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors"
output_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors"
os.makedirs(output_dir, exist_ok=True)

# === Define File-to-Label Mapping ===
factor_files = {
    "3F+MoM_daily.csv": "daily_ff3_mom",
    "5F+MoM_daily.csv": "daily_ff5_mom",
    "FFResearchF_monthly.csv": "monthly_ff5_mom",
    "P+SLiquidityfactors.csv": "monthly_ps_liquidity",
    "riskfreerate_daily.csv": "daily_riskfree",
    "vix_daily.csv": "daily_vix"
}

# === Column Renaming ===
column_renames = {
    "3F+MoM_daily.csv": ['date', 'ff3_mkt_rf', 'ff3_smb', 'ff3_hml', 'ff3_rf', 'ff3_mom'],
    "5F+MoM_daily.csv": ['date', 'ff5_mkt_rf', 'ff5_smb', 'ff5_hml', 'ff5_rmw', 'ff5_cma', 'ff5_rf', 'ff5_mom'],
    "FFResearchF_monthly.csv": ['date', 'ff5_mkt_rf', 'ff5_smb', 'ff5_hml', 'ff5_rmw', 'ff5_cma', 'ff5_rf', 'ff5_mom'],
    "P+SLiquidityfactors.csv": ['date', 'ps_level', 'ps_innov', 'ps_vwf'],
    "riskfreerate_daily.csv": ['date', 'risk_free'],
    "vix_daily.csv": ['date', 'vix_open', 'vix_high', 'vix_low', 'vix_close',
                      'vxx_open', 'vxoh', 'vxol', 'vxn', 'vxno', 'vxnh', 'vxnl', 'vxnd',
                      'vxdo', 'vxdh', 'vxdn', 'vxdl']
}

# === Process Files ===
daily_dfs = []

for filename, label in factor_files.items():
    filepath = os.path.join(input_dir, filename)

    try:
        df = pd.read_csv(filepath)
        if filename in column_renames:
            df.columns = column_renames[filename]
        
        df['date'] = pd.to_datetime(df['date'], errors='coerce')
        df = df.dropna(subset=['date']).sort_values('date')

        # Save renamed
        renamed_path = os.path.join(output_dir, f"{label}_renamed.csv")
        df.to_csv(renamed_path, index=False)
        print(f"✅ Saved: {renamed_path}")

        # If daily, keep for merging
        if label.startswith("daily_"):
            daily_dfs.append(df)

    except Exception as e:
        print(f"❌ Error with {filename}: {e}")

# === Merge Daily Datasets ===
if daily_dfs:
    merged_daily = reduce(lambda left, right: pd.merge(left, right, on='date', how='outer'), daily_dfs)
    merged_daily = merged_daily.sort_values("date")
    merged_path = os.path.join(output_dir, "merged_daily_factors.csv")
    merged_daily.to_csv(merged_path, index=False)
    print(f"\n🧩 Merged daily dataset saved to: {merged_path} ({merged_daily.shape[0]} rows, {merged_daily.shape[1]} columns)")
else:
    print("⚠️ No daily datasets merged.")


✅ Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/daily_ff3_mom_renamed.csv
✅ Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/daily_ff5_mom_renamed.csv
✅ Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/monthly_ff5_mom_renamed.csv
✅ Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/monthly_ps_liquidity_renamed.csv
❌ Error with riskfreerate_daily.csv: Length mismatch: Expected axis has 6 elements, new values have 2 elements
✅ Saved: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/daily_vix_renamed.csv

🧩 Merged daily dataset saved to: /Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/factors/merged_daily_factors.csv (2790 rows, 29 columns)


In [75]:
import pandas as pd
import os

# Input files
portfolio_file = "5x5ResearchPortfolios.csv"
liquidity_file = "monthly_ps_liquidity_renamed.csv"
output_file = "merged_monthly_factors.csv"

# Load 5x5 Research Portfolios (monthly)
df_portfolio = pd.read_csv(portfolio_file)
df_portfolio.columns = [col.lower() for col in df_portfolio.columns]
df_portfolio['date'] = pd.to_datetime(df_portfolio['date' if 'date' in df_portfolio.columns else 'DATE'])
df_portfolio = df_portfolio.sort_values('date')

# Load P+S Liquidity Factors (monthly, already renamed)
df_liquidity = pd.read_csv(liquidity_file)
df_liquidity['date'] = pd.to_datetime(df_liquidity['date'])
df_liquidity = df_liquidity.sort_values('date')

# Merge on 'date'
merged = pd.merge(df_portfolio, df_liquidity, on='date', how='inner')

# Save merged file
merged.to_csv(output_file, index=False)
print(f"✅ Saved merged monthly dataset to: {output_file}")


✅ Saved merged monthly dataset to: merged_monthly_factors.csv


In [76]:
import pandas as pd

# Load the already saved merged file
merged = pd.read_csv("merged_monthly_factors.csv")

# Reorder columns to put 'date' first
cols = ['date'] + [col for col in merged.columns if col != 'date']
merged = merged[cols]

# Overwrite the original file with corrected column order
merged.to_csv("merged_monthly_factors.csv", index=False)
print("✅ Updated file with 'date' as first column.")


✅ Updated file with 'date' as first column.


In [77]:
import pandas as pd

# Load the file
df = pd.read_csv("FF5Portfolios_dailyRet.csv")

# Rename columns for clarity
column_renames = {
    "smlo_vwret": "ret_vw_small_loBM",
    "smme_vwret": "ret_vw_small_medBM",
    "smhi_vwret": "ret_vw_small_hiBM",
    "bilo_vwret": "ret_vw_big_loBM",
    "bime_vwret": "ret_vw_big_medBM",
    "bihi_vwret": "ret_vw_big_hiBM",
    "smlo_ewret": "ret_ew_small_loBM",
    "smme_ewret": "ret_ew_small_medBM",
    "smhi_ewret": "ret_ew_small_hiBM",
    "bilo_ewret": "ret_ew_big_loBM",
    "bime_ewret": "ret_ew_big_medBM",
    "bihi_ewret": "ret_ew_big_hiBM",
    "smlo_nfirms": "nfirms_small_loBM",
    "smme_nfirms": "nfirms_small_medBM",
    "smhi_nfirms": "nfirms_small_hiBM",
    "bilo_nfirms": "nfirms_big_loBM",
    "bime_nfirms": "nfirms_big_medBM",
    "bihi_nfirms": "nfirms_big_hiBM",
    "smlo_msize": "avg_mktcap_small_loBM",
    "smme_msize": "avg_mktcap_small_medBM",
    "smhi_msize": "avg_mktcap_small_hiBM",
    "bilo_msize": "avg_mktcap_big_loBM",
    "bime_msize": "avg_mktcap_big_medBM",
    "bihi_msize": "avg_mktcap_big_hiBM",
    "smlo_level": "index_level_small_loBM",
    "smme_level": "index_level_small_medBM",
    "smhi_level": "index_level_small_hiBM",
    "bilo_level": "index_level_big_loBM",
    "bime_level": "index_level_big_medBM",
    "bihi_level": "index_level_big_hiBM",
}

# Apply renaming and fix date column
df.rename(columns=column_renames, inplace=True)
df["date"] = pd.to_datetime(df["date"])
df = df.sort_values("date")

# Save cleaned version
df.to_csv("FF5Portfolios_dailyRet_renamed.csv", index=False)
print("✅ Saved: FF5Portfolios_dailyRet_renamed.csv")


✅ Saved: FF5Portfolios_dailyRet_renamed.csv


In [78]:
import pandas as pd

# === File paths ===
daily_path = 'merged_daily_factors.csv'
monthly_path = 'merged_monthly_factors.csv'

# === Column renaming maps ===

daily_rename = {
    'mktrf': 'ff5_mkt_rf',
    'smb': 'ff5_smb',
    'hml': 'ff5_hml',
    'rmw': 'ff5_rmw',
    'cma': 'ff5_cma',
    'rf': 'ff5_rf',
    'umd': 'ff5_mom'
}

monthly_rename = {
    'mktrf': 'ff5_mkt_rf',
    'smb': 'ff5_smb',
    'hml': 'ff5_hml',
    'rmw': 'ff5_rmw',
    'cma': 'ff5_cma',
    'rf': 'ff5_rf',
    'umd': 'ff5_mom'
    # No changes to ps_level, ps_innov, ps_vwf
}

# === Apply changes to daily file ===
df_daily = pd.read_csv(daily_path)
df_daily = df_daily.rename(columns=daily_rename)
df_daily.to_csv(daily_path, index=False)
print(f"Updated column names in {daily_path}")

# === Apply changes to monthly file ===
df_monthly = pd.read_csv(monthly_path)
df_monthly = df_monthly.rename(columns=monthly_rename)
df_monthly.to_csv(monthly_path, index=False)
print(f"Updated column names in {monthly_path}")


Updated column names in merged_daily_factors.csv
Updated column names in merged_monthly_factors.csv


In [80]:
import pandas as pd
import re

# Load the original merged file
df = pd.read_csv("merged_monthly_factors.csv")

def rename_column(col):
    if col == "date":
        return "date"
    
    match = re.match(r's(\d)b(\d)_(vwret|ewret|nfirms|msize|level)', col)
    if not match:
        return col  # Leave unchanged if it doesn't match expected pattern

    size, bm, attr = match.groups()

    attr_map = {
        "vwret": "vw_ret",
        "ewret": "ew_ret",
        "nfirms": "n_firms",
        "msize": "avg_mktcap",
        "level": "index_level"
    }

    return f"{attr_map[attr]}_size{size}_bm{bm}"

# Rename all columns
df.columns = [rename_column(col) for col in df.columns]

# Save the result
df.to_csv("merged_monthly_factors_renamed.csv", index=False)
print("✅ Saved: merged_monthly_factors_renamed.csv with clearer column names.")


✅ Saved: merged_monthly_factors_renamed.csv with clearer column names.


In [81]:
import pandas as pd

# Load datasets
merged_df = pd.read_csv("merged_daily_factors.csv", parse_dates=["date"])
rf_df = pd.read_csv("riskfreerate_daily.csv", parse_dates=["date"])

# Normalize the date format in rf_df to match merged_df
rf_df['date'] = pd.to_datetime(rf_df['date']).dt.strftime('%Y-%m-%d')
merged_df['date'] = pd.to_datetime(merged_df['date']).dt.strftime('%Y-%m-%d')

# Rename riskfree columns to clearly show their origin
rf_df = rf_df.rename(columns={
    'mktrf': 'riskfree_mktrf',
    'smb': 'riskfree_smb',
    'hml': 'riskfree_hml',
    'rf': 'riskfree_rf',
    'umd': 'riskfree_umd'
})

# Merge the dataframes
merged_with_rf = pd.merge(merged_df, rf_df, on='date', how='left')

# Save to CSV
merged_with_rf.to_csv("merged_daily_factors_with_riskfree.csv", index=False)

print("Merged file saved as 'merged_daily_factors_with_riskfree.csv'")


Merged file saved as 'merged_daily_factors_with_riskfree.csv'


In [82]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/fundamentals/")

In [83]:
import os
import pandas as pd

# Directory where your fundamental files are located
input_dir = "/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/fundamentals/"
output_dir = "fundamentals_cleaned"
os.makedirs(output_dir, exist_ok=True)

# Loop over each file in the directory
for file in os.listdir(input_dir):
    if file.endswith("_monthly_valuation_measures.csv"):
        filepath = os.path.join(input_dir, file)
        df = pd.read_csv(filepath, index_col=0)

        # Transpose and clean
        df = df.transpose().reset_index()
        df = df.rename(columns={"index": "date"})

        # Optional: Add ticker column from filename (e.g., AAPL from AAPL_monthly_valuation_measures.csv)
        ticker = file.split("_")[0]
        df.insert(1, "ticker", ticker)

        # Save the cleaned file
        output_filepath = os.path.join(output_dir, file.replace(".csv", "_cleaned.csv"))
        df.to_csv(output_filepath, index=False)

        print(f"Processed {file} -> {output_filepath}")


Processed GOOG_monthly_valuation_measures.csv -> fundamentals_cleaned/GOOG_monthly_valuation_measures_cleaned.csv
Processed GOOGL_monthly_valuation_measures.csv -> fundamentals_cleaned/GOOGL_monthly_valuation_measures_cleaned.csv
Processed NVDA_monthly_valuation_measures.csv -> fundamentals_cleaned/NVDA_monthly_valuation_measures_cleaned.csv
Processed TSLA_monthly_valuation_measures.csv -> fundamentals_cleaned/TSLA_monthly_valuation_measures_cleaned.csv
Processed META_monthly_valuation_measures.csv -> fundamentals_cleaned/META_monthly_valuation_measures_cleaned.csv
Processed BRK-B_monthly_valuation_measures.csv -> fundamentals_cleaned/BRK-B_monthly_valuation_measures_cleaned.csv
Processed AVGO_monthly_valuation_measures.csv -> fundamentals_cleaned/AVGO_monthly_valuation_measures_cleaned.csv
Processed AMZN_monthly_valuation_measures.csv -> fundamentals_cleaned/AMZN_monthly_valuation_measures_cleaned.csv
Processed AAPL_monthly_valuation_measures.csv -> fundamentals_cleaned/AAPL_monthly_v

In [97]:
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 150)

# Load all datasets
daily_factors = pd.read_csv("merged_daily_factors_final.csv", parse_dates=['date'])
technicals = pd.read_csv("/v1/AAPL_2014_2024_technical_cleaned.csv", parse_dates=['date'])
valuation = pd.read_csv("/v1/AAPL_monthly_valuation_measures_cleaned.csv", parse_dates=['date'])
options = pd.read_csv("/v1/AAPL_options_all.csv", parse_dates=['date'])



FileNotFoundError: [Errno 2] No such file or directory: 'merged_daily_factors_final.csv'

In [101]:
####### Valuation EDA  to turn monthly valuations files in the final clean valuation files by ticker

In [103]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/fundamentals/fundamentals_cleaned")

In [132]:
# Load CSV and skip first row
valuation = pd.read_csv("TSLA_monthly_valuation_measures_cleaned.csv")


# Drop the first data row if it contains 'ttm' (non-date value)
valuation = valuation[~valuation["date"].str.lower().str.contains("ttm", na=False)]

# Drop undesired columns explicitly
valuation = valuation.drop(columns=["EnterprisesValueRevenueRatio", "EnterprisesValueEBITDARatio"], errors='ignore')


# Convert 'date' to datetime
valuation["date"] = pd.to_datetime(valuation["date"], errors="coerce")
valuation = valuation.dropna(subset=["date"])

# Clean numeric fields
for col in valuation.columns:
    if col not in ["date", "ticker"]:
        valuation[col] = (
            valuation[col].astype(str)
            .str.replace(",", "", regex=False)
            .replace("", pd.NA)
            .astype(float)
        )

# Normalize and filter date
valuation["date"] = valuation["date"].dt.normalize()
valuation = (
    valuation.sort_values("date")
             .drop_duplicates(subset="date", keep="last")
)
valuation = valuation[
    (valuation["date"] >= "2014-01-01") & (valuation["date"] <= "2024-12-31")
]


print("✅ Valuation data cleaned, trimmed, and ready.")


✅ Valuation data cleaned, trimmed, and ready.


In [133]:
valuation.to_csv("TSLA_valuations.csv")

In [134]:
#### Options EDA 

In [137]:
os.chdir("/Users/auguste/Desktop/Dossiers/HEC/Courses/RP/rp-adaptive-ml-trading/data/raw/raw/options/options_cleaned_from_xlsx")

In [144]:
import pandas as pd
import numpy as np
from dateutil import parser

# --- Load and clean raw options data ---
opt = pd.read_csv("BRK-B_options_all.csv")

# Rename key columns
rename_map = {
    'last_trade_date_(edt)': 'last_trade_date',
    'expiry_date': 'expiry_date',
    'contract_name': 'contract_name',
    'open_interest': 'open_interest',
    'volume': 'volume',
    'implied_volatility': 'implied_volatility'
}
opt.rename(columns=rename_map, inplace=True)

# Normalize column names
opt.columns = (
    opt.columns
    .str.strip()
    .str.lower()
    .str.replace(r"[^\w]+", "_", regex=True)
    .str.replace(r"_+", "_", regex=True)
    .str.strip("_")
)

# --- Robust date parsing ---
def parse_mixed_datetime(val):
    if pd.isna(val):
        return pd.NaT
    try:
        return pd.to_datetime(val, format='%m/%d/%Y %I:%M %p')
    except Exception:
        try:
            return pd.to_datetime(val)
        except Exception:
            return pd.NaT

opt['last_trade_date'] = opt['last_trade_date'].apply(parse_mixed_datetime)
opt['expiry_date'] = pd.to_datetime(opt['expiry_date'], errors='coerce')

# Filter invalid rows
opt = opt.dropna(subset=['last_trade_date', 'expiry_date'])
opt = opt[opt['contract_name'].str.contains("BRKB", na=False)]

# --- Numeric conversion ---
opt['open_interest'] = pd.to_numeric(opt['open_interest'], errors='coerce')
opt['volume'] = pd.to_numeric(opt['volume'], errors='coerce')
opt['implied_volatility'] = (
    pd.to_numeric(opt['implied_volatility'].astype(str).str.replace('%', ''), errors='coerce') / 100
)

# --- Feature engineering ---
opt['option_type'] = opt['contract_name'].str.extract(r'(\d+[CP])')[0].str[-1].map({'C': 'call', 'P': 'put'})
opt['dte'] = (opt['expiry_date'] - opt['last_trade_date']).dt.days
opt['date'] = opt['last_trade_date'].dt.floor('D')

# Define DTE bucket logic
def tag_dte_bucket(dte):
    if 0 <= dte <= 10:
        return 'short'
    elif 20 <= dte <= 40:
        return 'medium'
    elif 50 <= dte <= 150:
        return 'long'
    return None

opt['dte_bucket'] = opt['dte'].apply(tag_dte_bucket)
opt = opt[opt['dte_bucket'].notnull()]



# Output preview
print("✅ Parsed and cleaned options data.")
print("📌 Aggregated features (long-form):")
print(opt.head())

opt.to_csv("BRK-B_options.csv")



✅ Parsed and cleaned options data.
📌 Aggregated features (long-form):
         contract_name     last_trade_date strike last_price   bid   ask change change  volume  open_interest  implied_volatility  expiry_sheet  \
0  BRKB250620C00190000 2025-06-16 09:32:00    190     300.61  0.00  0.00   0.00  0.00%  5.0000         0.0000              0.0000  Jun 20, 2025   
1  BRKB250620C00200000 2025-06-13 14:42:00    200     286.81  0.00  0.00   0.00  0.00% 10.0000         0.0000              0.0000  Jun 20, 2025   
2  BRKB250620C00210000 2025-02-06 14:30:00    210     292.22  0.00  0.00   0.00  0.00% 10.0000         0.0000              0.0000  Jun 20, 2025   
3  BRKB250620C00220000 2025-03-06 09:42:00    220     282.07  0.00  0.00   0.00  0.00% 29.0000         0.0000              0.0000  Jun 20, 2025   
4  BRKB250620C00230000 2025-03-06 09:46:00    230     271.33  0.00  0.00   0.00  0.00%  7.0000         0.0000              0.0000  Jun 20, 2025   

  expiry_date option_type  dte       date dte_b